In [15]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import librosa
import os
import shutil
from concurrent.futures import ThreadPoolExecutor
import soundfile as sf
import numpy as np

In [17]:
SAMPLE_RATE = 16000

In [ ]:
input_path="/content/drive/MyDrive/USP/qualificacao/EXPERIMENTOS/podcast_audio_only_one/"

In [ ]:
output_path="/content/drive/MyDrive/USP/qualificacao/EXPERIMENTOS/output_path/"

In [15]:
! apt install ffmpeg


Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.


In [ ]:
! pip install inaSpeechSegmenter

In [23]:
import os
from concurrent.futures import ThreadPoolExecutor
import csv
import pandas as pd
from pydub import AudioSegment

from inaSpeechSegmenter import  Segmenter
from inaSpeechSegmenter.export_funcs import seg2csv, seg2textgrid


In [24]:
seg = Segmenter()

# Segmentação e Trimagem - Áudios com 1 locutor

In [33]:
class SegmentationAudio:


    def __init__(self,audio_directory,save_directory,time_seconds,sample_rate):

        self.audio_directory = audio_directory
        self.save_directory = save_directory
        self.time_seconds = time_seconds
        self.sample_rate = sample_rate

    def process_audio_file(self,file):

        segmentation = seg(file)
        filename = os.path.basename(file)
        filename = os.path.splitext(filename)[0]
        rows = []
        for row in segmentation:
            res = (filename,) + tuple(row)
            # print(res)
            rows.append(res)

        return rows

    def process_audio_files(self,input_dir, output_file):
        with ThreadPoolExecutor() as executor:
            files = []
            for dirpath, dirnames, filenames in os.walk(input_dir):
                for filename in filenames:
                    if filename.endswith('.ogg'):
                        files.append(os.path.join(dirpath, filename))
            all_rows = []
            for rows in executor.map(self.process_audio_file, files):
                print(rows)
                all_rows.extend(rows)

            with open(output_file, 'w+', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['Filename','genre','start_time','end_time'])
                writer.writerows(all_rows)


    def read_csv_info(self,file_csv,input_dir):
        list_all_genres=[]
        df = pd.read_csv(file_csv,delimiter=',')
        for index,row in df.iterrows():
             audio_filename = row['Filename']
             genre = row['genre']
             start_time=float(row['start_time'])
             end_time=float(row['end_time'])

             for dirpath,dirnames, filenames in os.walk(input_dir):
                for filename in filenames:
                     audio_path = os.path.join(dirpath,filename)
                     audio = AudioSegment.from_file(audio_path)
                     if genre in ['male','female']:
                         trimmed_audio = audio[start_time * 1000:end_time * 1000]
                         trimmed_filename = f"trimmed_{audio_filename}"
                         trimmed_filepath = os.path.join(dirpath, trimmed_filename)
                         trimmed_audio.export(trimmed_filepath, format='wav')
                         #list_all_genres.append(trimmed_audio)
                         print(f"SALVAR AQUI'{trimmed_filepath}'.")

        #combined_audio = AudioSegment.empty()
        #for audio_segment in list_all_genres:
              #combined_audio += audio_segment

        #combined_audio.export("combined_audio.wav", format="wav")
        #print("AQUI.")


    def trim_audio_files(self):
        for dirpath, dirnames, filenames in os.walk(self.audio_directory):
            for i, filename in enumerate(filenames):
                audio_path = os.path.join(dirpath, filename)
                audio_ = AudioSegment.from_file(audio_path)
                num_segments = len(audio_) // (self.time_seconds * 1000)
                dir_initial = os.path.basename(dirpath)
                filename = filename.replace(".ogg","")
                for segment in range(num_segments):
                    start_time = segment * self.time_seconds * 1000
                    end_time = (segment + 1) * self.time_seconds * 1000
                    trimmed_audio = audio_[start_time:end_time]
                    resampled_audio = trimmed_audio.set_frame_rate(self.sample_rate)
                    mono_audio = resampled_audio.set_channels(1)
                    output_trimmed_audio = os.path.join(self.save_directory, f"{dir_initial}_{filename}_{segment}.wav")
                    mono_audio.export(output_trimmed_audio, format="wav")

In [34]:

#output = 'my_csv_file.csv'
audio_directory = "/content/drive/MyDrive/USP/qualificacao/EXPERIMENTOS/podcast_audio_only_one/"
save_directory = "/content/drive/MyDrive/USP/qualificacao/EXPERIMENTOS/teste/"
time_seconds = 10
sample_rate = 16000
seg_ = SegmentationAudio(audio_directory,save_directory,time_seconds,sample_rate)
seg_.trim_audio_files()

#Remoção de Musica e silêncio

In [18]:
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install spleeter

In [20]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
from pydub import AudioSegment
from spleeter.separator import Separator
import os
import shutil
from tqdm import tqdm

In [22]:
class CleaningAudiosDataset(object):

    def __init__(self,path_with_folders,save_local_folder,batch_size,destination_path):
        self.path_with_folders = path_with_folders
        self.save_local_folder = save_local_folder
        self.batch_size = batch_size
        self.destination_path = destination_path

    def separate_voice_from_music(self):

        audio_files = [os.path.join(self.path_with_folders, f) for f in os.listdir(self.path_with_folders) if f.endswith('.wav')]

        batches = [audio_files[i:i + self.batch_size] for i in range(0, len(audio_files), self.batch_size)]
        separator = Separator('spleeter:2stems')

        for batch in tqdm(batches):
            for audio_file in batch:
                prediction = separator.separate_to_file(audio_file, self.save_local_folder)

    def rename_and_copy(self,path):
        for dirpath,dirname,filenames in os.walk(path):
            for i, filename in enumerate(filenames):
                audio_file = os.path.join(dirpath,filename)
                if  not (audio_file.endswith('accompaniment.wav')):
                    dir_name = os.path.basename(dirpath)
                    file_name = os.path.basename(filename)
                    new_file = os.path.join(dirpath,f"{dir_name}.wav")
                    #os.rename(audio_file, new_file)
                    new_path = os.path.join(self.destination_path, f"{dir_name}.wav")
                    shutil.move(audio_file, new_path)


    def remove_low_volume(self,threshold):
        for file in os.listdir(self.destination_path):
            if file.endswith('.wav'):
                sound = AudioSegment.from_file(os.path.join(self.destination_path,file),format='wav')
                loud = [seg for seg in sound if seg.dBFS > threshold]
                if loud:
                    new = loud[0]
                    for l in loud[1:]:
                        new+=l

                    output = os.path.join(self.destination_path,file)
                    new.export(output,format='wav')
                else:
                    print("Low loud")


    def remove_44bytes_audio_files(self,destination_path):

        for dirpath,dirname,filenames in os.walk(destination_path):
            for i,filename in enumerate(filenames):
                audio_file = os.path.join(dirpath,filename)
                if os.path.isfile(audio_file) and os.path.getsize(audio_file) == 44:
                    print(f"Removendo arquivo de 44bytes...em 1,2,3:{filename}")
                    os.remove(audio_file)

    def get_capital_begin_letter(self,audio):
        capital_letter = audio.split('_')[0]
        return capital_letter

    def separate_audio_per_folder(self,destination_path):
        audio_files = [f for f in os.listdir(destination_path) if f.endswith('.wav')]
        for audio in audio_files:
            capital_letter = self.get_capital_begin_letter(audio)
            capital_folder = os.path.join(destination_path,capital_letter)
            os.makedirs(capital_folder,exist_ok=True)
            audio_path = os.path.join(destination_path, audio)
            new_audio_path = os.path.join(capital_folder, audio)
            os.rename(audio_path, new_audio_path)

    def process_audio_files(self):
        self.separate_voice_from_music()
        self.rename_and_copy(self.save_local_folder)
        self.remove_low_volume(threshold=0)

    def main(self):
        self.process_audio_files()


In [23]:
cleaner = CleaningAudiosDataset(path_with_folders='/content/drive/MyDrive/USP/qualificacao/EXPERIMENTOS/teste/',
                                save_local_folder='/content/drive/MyDrive/USP/qualificacao/EXPERIMENTOS/save_local/',
                                batch_size=64,
                                destination_path='/content/drive/MyDrive/USP/qualificacao/EXPERIMENTOS/destination_path/')
#cleaner.main()

In [24]:
cleaner.remove_44bytes_audio_files(destination_path='/content/drive/MyDrive/USP/qualificacao/EXPERIMENTOS/destination_path/')

# Separando os arquivos nos diretórios específicos - Teste


In [11]:
destination_path='/content/drive/MyDrive/USP/qualificacao/EXPERIMENTOS/destination_path/'

In [12]:
import os

audio_folder = destination_path
audio_files = [f for f in os.listdir(audio_folder) if f.endswith('.wav')]

In [13]:
def extrair_nome_capital(audio_file):
    nome_capital = audio_file.split('_')[0]
    return nome_capital


In [14]:
for audio_file in audio_files:
    capital = extrair_nome_capital(audio_file)
    capital_folder = os.path.join(audio_folder, capital)
    os.makedirs(capital_folder, exist_ok=True)
    audio_path = os.path.join(audio_folder, audio_file)
    new_audio_path = os.path.join(capital_folder, audio_file)
    os.rename(audio_path, new_audio_path)
